# Set parametized values

In [0]:
dbutils.widgets.text('folder_name','')

In [0]:
folder_name = dbutils.widgets.get('folder_name')

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.readStream.format('delta')\
    .load(f'abfss://bronze@osspstorageaccount.dfs.core.windows.net/{folder_name}')

In [0]:
folder_name = folder_name.replace('/','')
surrogate_key_col = f"sg_{folder_name}key"
df = df.withColumn(surrogate_key_col, lit('None'))\
    .withColumn('updated_at', current_timestamp())

# Customer transformation

In [0]:
if folder_name == 'customer':
    df = df.withColumn('Name',upper(col('Name')))

# Date Transformation

# Paymentmethod Transformation

In [0]:
if folder_name == 'paymentmethod':
    # upper payment type, provider, isdigitalwallet
    df = df.withColumn('PaymentType',upper(col('PaymentType')))\
        .withColumn('Provider',upper(col('Provider')))\
        .withColumn('IsDigitalWallet',upper(col('IsDigitalWallet')))


# Product Transformation

In [0]:
if folder_name == 'product':
    df = df.withColumn('ProductName',upper(col('ProductName')))\
        .withColumn('Brand',upper(col('Brand')))

# Promotion Transformation

In [0]:
if folder_name == 'promotion':
    df = df.withColumn('PromoCode',upper(col('PromoCode')))
    df = df.withColumn('Durationdays', datediff(col('EndDate'),col('StartDate')))

# Region Transformation

In [0]:
if folder_name == 'region':
    df = df.withColumn('Country',upper(col('Country')))\
        .withColumn('State',upper(col('State')))\
        .withColumn('City',upper(col('City')))\
        .withColumn('EconomicZone',upper(col('EconomicZone')))


# Shippingmethod transformation

In [0]:
if folder_name == 'shippingmethod':
    df = df.withColumn('Method',upper(col('Method')))\
        .withColumn('Carrier', upper(col('Carrier')))

# Factsales transformation

In [0]:
df.writeStream.format('delta')\
    .option('checkpointLocation',f'abfss://silver@osspstorageaccount.dfs.core.windows.net/checkpoint/{folder_name}')\
    .outputMode('append')\
    .trigger(once=True)\
    .start(f'abfss://silver@osspstorageaccount.dfs.core.windows.net/{folder_name}')